## 上一个代码的输出

In [22]:
output_pre = {'args': {'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']}, 'question_types': ['check_disease']}

In [23]:
output_pre

{'args': {'肺气肿': ['disease'], '百日咳': ['disease'], '血常规': ['check']},
 'question_types': ['check_disease']}

In [24]:
args = output_pre['args']

In [25]:
entity_dict = {}
for arg, types in args.items():
    for type in types:
        if type not in entity_dict:
            entity_dict[type] = [arg]
        else:
            entity_dict[type].append(arg)

In [26]:
entity_dict

{'disease': ['肺气肿', '百日咳'], 'check': ['血常规']}

In [27]:
question_types = output_pre['question_types']

In [28]:
question_types

['check_disease']

In [29]:
question_type

'check_disease'

In [10]:
sqls = []

In [16]:
def sql_transfer(question_type, entities):
    '''
    不同的提问意图，对应不同关系的cypher查询语句
    '''
    
    if not entities:
        return []

    # 查询语句
    sql = []
    # 查询疾病的病因
    if question_type == 'disease_cause':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cause".format(i) for i in entities]

    # 查询疾病的预防措施
    elif question_type == 'disease_prevent':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.prevent".format(i) for i in entities]

    # 查询疾病的疗程
    elif question_type == 'disease_lasttime':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_lasttime".format(i) for i in entities]

    # 查询疾病的治愈率
    elif question_type == 'disease_cureprob':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cured_prob".format(i) for i in entities]

    # 查询疾病的疗法
    elif question_type == 'disease_cureway':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.cure_way".format(i) for i in entities]

    # 查询疾病的易感人群
    elif question_type == 'disease_easyget':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.easy_get".format(i) for i in entities]

    # 查询疾病的描述
    elif question_type == 'disease_desc':
        sql = ["MATCH (m:Disease) where m.name = '{0}' return m.name, m.desc".format(i) for i in entities]

    # 查询疾病的症状
    elif question_type == 'disease_symptom':
        sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询症状对应的疾病
    elif question_type == 'symptom_disease':
        sql = ["MATCH (m:Disease)-[r:has_symptom]->(n:Symptom) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询疾病的并发症
    elif question_type == 'disease_acompany':
        sql1 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2
    # 查询疾病的忌口
    elif question_type == 'disease_not_food':
        sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 查询疾病建议吃的东西
    elif question_type == 'disease_do_food':
        sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 已知忌口查疾病
    elif question_type == 'food_not_disease':
        sql = ["MATCH (m:Disease)-[r:no_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 已知推荐查疾病
    elif question_type == 'food_do_disease':
        sql1 = ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 查询疾病常用药品－药品别名记得扩充
    elif question_type == 'disease_drug':
        sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2

    # 已知药品查询能够治疗的疾病
    elif question_type == 'drug_disease':
        sql1 = ["MATCH (m:Disease)-[r:common_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql2 = ["MATCH (m:Disease)-[r:recommand_drug]->(n:Drug) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]
        sql = sql1 + sql2
    # 查询疾病应该做的检查
    elif question_type == 'disease_check':
        sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where m.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    # 已知检查查询疾病
    elif question_type == 'check_disease':
        sql = ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '{0}' return m.name, r.name, n.name".format(i) for i in entities]

    return sql

In [39]:
sql_transfer('check_disease', ['肺气肿', '百日咳'])

["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '肺气肿' return m.name, r.name, n.name",
 "MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '百日咳' return m.name, r.name, n.name"]

In [48]:
sqls = []
for question_type in question_types:
    sql_ = {}
    sql_['question_type'] = question_type
    sql_['sql'] = sql_transfer(question_type, entity_dict.get(question_type.split('_')[0]))
    sqls.append(sql_)

In [49]:
sqls

[{'question_type': 'check_disease',
  'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]

In [47]:
question_type

'check_disease'

In [17]:
# for question_type in question_types:
#     sql_ = {}
#     sql_['question_type'] = question_type
#     sql = []
#     if question_type == 'disease_symptom':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'symptom_disease':
#         sql = sql_transfer(question_type, entity_dict.get('symptom'))

#     elif question_type == 'disease_cause':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_acompany':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_not_food':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_do_food':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'food_not_disease':
#         sql = sql_transfer(question_type, entity_dict.get('food'))

#     elif question_type == 'food_do_disease':
#         sql = sql_transfer(question_type, entity_dict.get('food'))

#     elif question_type == 'disease_drug':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'drug_disease':
#         sql = sql_transfer(question_type, entity_dict.get('drug'))

#     elif question_type == 'disease_check':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'check_disease':
#         sql = sql_transfer(question_type, entity_dict.get('check'))

#     elif question_type == 'disease_prevent':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_lasttime':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_cureway':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_cureprob':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_easyget':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     elif question_type == 'disease_desc':
#         sql = sql_transfer(question_type, entity_dict.get('disease'))

#     if sql:
#         sql_['sql'] = sql

#         sqls.append(sql_)

In [18]:
sqls

[{'question_type': 'check_disease',
  'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]

In [50]:
sqls

[{'question_type': 'check_disease',
  'sql': ["MATCH (m:Disease)-[r:need_check]->(n:Check) where n.name = '血常规' return m.name, r.name, n.name"]}]